# Transfer Learning


In [1]:
import os
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from os import getcwd

In [2]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

--2021-11-28 04:05:37--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.120.128, 142.250.128.128, 142.251.6.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.120.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M   200MB/s    in 0.4s    

2021-11-28 04:05:37 (200 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]



In [3]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

local_weights = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
pre_trained_model = InceptionV3(input_shape=(200, 200, 3),
                                include_top=False,
                                weights=None)
pre_trained_model.load_weights(local_weights)
for layer in pre_trained_model.layers:
    layer.trainable = False
pre_trained_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 200, 200, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 99, 99, 32)   864         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 99, 99, 32)  96          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                       

In [4]:
last_layer = pre_trained_model.get_layer('mixed7')
print('Last Layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

Last Layer output shape:  (None, 10, 10, 768)


In [5]:
class MyCallbacks(tf.keras.callbacks.Callback):
    count=0
    def on_epoch_end(self, epoch, logs):

        if logs['accuracy'] > 0.99:
            self.count+=1
            if self.count==5:
                print('/nEarly Stopping the training as Validation Aaccuracy exceeded 99%!')
                self.model.stop_training = True
        else:
            self.count=0  

In [6]:
callbacks = MyCallbacks()

In [7]:
from tensorflow.keras.optimizers import RMSprop

x = layers.Flatten()(last_output)

# 128 neuron in the fully-connected layer
x=layers.Dense(128, activation='relu')(x)
# sigmoid activation
x=layers.Dense(1, activation=tf.nn.sigmoid)(x)


model = Model(pre_trained_model.input, x)

model.compile(optimizer=RMSprop(lr=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 200, 200, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 99, 99, 32)   864         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 99, 99, 32)  96          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                              

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


# Now just load the data and train as you would

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import glob
from random import *
from tensorflow.keras.utils import to_categorical
d = {}
with open('/content/drive/MyDrive/Handwriting plagiarism/forms_for_parsing.txt') as f:
    for line in f:
        key = line.split(' ')[0]
        writer = line.split(' ')[1]
        d[key] = writer
tmp = []
target_list = []
# All images will be rescaled by 1./255
batch_size= 10
# train_datagen = ImageDataGenerator(rescale=1/255,horizontal_flip=True,vertical_flip=True)
# val_datagen = ImageDataGenerator(rescale=1/255)
# Create array of file names and corresponding target writer names
path_to_files = os.path.join('/content/drive/MyDrive/Handwriting plagiarism/dataset/', '*')
for filename in sorted(glob.glob(os.path.join(path_to_files, '*'))):
    for fn in sorted(glob.glob(os.path.join(filename, '*'))):
        # print(fn)
        tmp.append(fn)
        image_name = filename.split('/')[-1]
        file, ext = os.path.splitext(image_name)
        parts = file.split('-')
        form = parts[0] + '-' + parts[1]
        for key in d:
            if key == form:
                target_list.append(str(d[form]))

img_files = np.asarray(tmp)
img_targets = np.asarray(target_list)


In [10]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(img_targets)
encoded_Y = encoder.transform(img_targets)

print(img_files[:5], img_targets[:5], encoded_Y[:5])

['/content/drive/MyDrive/Handwriting plagiarism/dataset/a01/a01-000u/a01-000u-s00-00.png'
 '/content/drive/MyDrive/Handwriting plagiarism/dataset/a01/a01-000u/a01-000u-s00-01.png'
 '/content/drive/MyDrive/Handwriting plagiarism/dataset/a01/a01-000u/a01-000u-s00-02.png'
 '/content/drive/MyDrive/Handwriting plagiarism/dataset/a01/a01-000u/a01-000u-s00-03.png'
 '/content/drive/MyDrive/Handwriting plagiarism/dataset/a01/a01-000u/a01-000u-s01-00.png'] ['000' '000' '000' '000' '000'] [0 0 0 0 0]


In [11]:
from PIL import Image
batch_size = 16
num_classes = 50
def generate_data(samples, target_files,  batch_size=batch_size, factor = 0.1 ):
    num_samples = len(samples)
    from sklearn.utils import shuffle
    while 1: # Loop forever so the generator never terminates
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]
            batch_targets = target_files[offset:offset+batch_size]

            images = []
            targets = []
            for i in range(len(batch_samples)):
                batch_sample = batch_samples[i]
                batch_target = batch_targets[i]
                im = Image.open(batch_sample)
                cur_width = im.size[0]
                cur_height = im.size[1]

                # print(cur_width, cur_height)
                height_fac = 113 / cur_height

                new_width = int(cur_width * height_fac)
                size = new_width, 113

                imresize = im.resize((size), Image.ANTIALIAS)  # Resize so height = 113 while keeping aspect ratio
                now_width = imresize.size[0]
                now_height = imresize.size[1]
                # Generate crops of size 113x113 from this resized image and keep random 10% of crops

                avail_x_points = list(range(0, now_width - 113 ))# total x start points are from 0 to width -113

                # Pick random x%
                pick_num = int(len(avail_x_points)*factor)

                # Now pick
                random_startx = sample(avail_x_points,  pick_num)

                for start in random_startx:
                    imcrop = imresize.crop((start, 0, start+113, 113))
                    images.append(np.asarray(imcrop))
                    targets.append(batch_target)

            # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(targets)

            #reshape X_train for feeding in later
            X_train = X_train.reshape(X_train.shape[0], 113, 113, 1)
            #convert to float and normalize
            X_train = X_train.astype('float32')
            X_train /= 255

            #One hot encode y
            y_train = to_categorical(y_train, num_classes)

            yield shuffle(X_train, y_train)

In [12]:
from sklearn.model_selection import train_test_split
train_files, rem_files, train_targets, rem_targets = train_test_split(
        img_files, encoded_Y, train_size=0.66, random_state=52, shuffle= True)

validation_files, test_files, validation_targets, test_targets = train_test_split(
        rem_files, rem_targets, train_size=0.5, random_state=22, shuffle=True)

print(train_files.shape, validation_files.shape, test_files.shape)
print(train_targets.shape, validation_targets.shape, test_targets.shape)


train_generator = generate_data(train_files, train_targets, batch_size=batch_size, factor = 0.3)
validation_generator = generate_data(validation_files, validation_targets, batch_size=batch_size, factor = 0.3)
test_generator = generate_data(test_files, test_targets, batch_size=batch_size, factor = 0.1)
print(type(train_generator))


(11074,) (2852,) (2853,)
(11074,) (2852,) (2853,)
<class 'generator'>


In [ ]:
# total_sample=train_generator.n
import numpy as np
from keras.preprocessing import image

nb_epoch = 5

samples_per_epoch = 25
nb_val_samples = 25


#Model fit generator
history = model.fit(train_generator, steps_per_epoch= samples_per_epoch,
                                     validation_data=validation_generator,
                                     validation_steps=nb_val_samples, 
                    epochs=nb_epoch, 
                    verbose=1 
                    )

# model.save('model.h5')

In [ ]:
acc=history.history['accuracy']
loss=history.history['loss']
val_acc=history.history['val_accuracy']
val_loss=history.history['val_loss']

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(acc,'b', label='accuracy')
plt.plot(val_acc,'lightblue', label='val_accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()
print("Accuracy: ", history.history['accuracy'][-1])
print("Validation Accuracy: ",history.history['val_accuracy'][-1])

In [ ]:
plt.plot(loss,'r', label='loss')
plt.plot(val_loss,'lightpink', label='val_loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()
print("Loss:",history.history['loss'][-1])
print("Validation Loss:",history.history['val_loss'][-1])